In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.utils import save_image
import os

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_size = 784
n_dim = 400
z_dim = 20
bs = 4
lr_rate = 0.002

In [ ]:
data = torchvision.datasets.MNIST(root = '../datasets/MNIST/',
                                    train=True,
                                     transform = torchvision.transforms.ToTensor(),
                                    download = True)
data_loader = torch.utils.data.DataLoader(dataset=data, batch_size=bs, shuffle = True)

In [ ]:
class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=400,z_dim=20):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(h_dim, z_dim)
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)
        
    def encode(self, x):
        out = F.relu(self.fc1(x))
        return self.fc2(out), self.fc3(out)
    
    def repara(self, mu, var):
        ###    Reparametrization trick to allow for gradients to pass during backprop
        st_dev = torch.exp(var/2)
        eps = torch.randn_like(st_dev)
        return mu + eps * st_dev
    
    def decode(self, x):
        out = F.relu(self.fc4(x))
        out = F.sigmoid(self.fc5(out))
        return out
    
    def forward(self, x):
        mu, var = self.encode(x)
        z = self.repara(mu, var)
        recont = self.decode(z)
        return recont, mu, var
    
model = VAE().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

In [ ]:
n_e = 5
for e in range(n_e):
    for i, (img, _) in enumerate(data_loader):
        
        img = img.to(device).view(-1, image_size)
        recon, mu, var = model(img)
        
        #         Loss is the sum of reconstruction loss and kl divergence
        
        loss_recon = F.binary_cross_entropy(recon, img)
        kldiv = 0.5*torch.sum(1 + var - mu.pow(2) - var.exp())
        loss_val = loss_recon + kldiv
        
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()
        
        if(i+1)%10==0:
            print("Epoch[{}/{}], step [{}/{}], loss ---recon--->{:.2f}, loss---kl--->{:.2f}".
                 format(e+1, n_e, i+1, len(data_loader), loss_recon.item(), kldiv.item()))
    
    ####   Test the model
    with torch.no_grad():
        # Save the sampled images
        z = torch.randn(bs, z_dim).to(device)
        out = model.decode(z).view(-1, 1, 28, 28)
        save_image(out, os.path.join('../datasets/vaesamples/', 'sampled_{}.png'.format(epoch+1)))

        # Save the reconstructed images
        out, _, _ = model(x)
        x_concat = torch.cat([x.view(-1, 1, 28, 28), out.view(-1, 1, 28, 28)], dim=3)
        save_image(x_concat, os.path.join('../datasets/vaesamples/', 'reconst_{}.png'.format(epoch+1)))